In [1]:
# Python script to do the analysis based on the account table
import openpyxl
import numpy as np
import pandas as pd
import os
from util import rs_conn, mail
import time
import datetime
import logging

In [2]:
# Get the redshift user name and password
rs_user = os.getenv('RS_USER')
rs_pass = os.getenv('RS_PASS')

# Connect to the Redshift
rs = rs_conn.RS_CONN(rs_user=rs_user, rs_pass=rs_pass)
rs.set_Conn()

In [3]:
acct_engagement_pull_all_cycle="""
select * from ua.eos_acct_summary
;
"""
all_df_1=rs.get_result_df(sql_query=acct_engagement_pull_all_cycle)

In [5]:
all_df_1['attrition_pct'] = all_df_1['attrition_cnt']/all_df_1['actived_acct_cnt']
all_df_1['co_pct'] = all_df_1['co_cnt']/ all_df_1['actived_acct_cnt']
all_df_1['retention_pct'] = all_df_1['open_acct_cnt']/ all_df_1['actived_acct_cnt']
all_df_1['revolving_pct'] = all_df_1['tot_adb_amt']/ all_df_1['all_adb_amt']

all_df_1['active_rate_2'] = all_df_1['active_acct_cnt']/all_df_1['actived_acct_cnt']

In [6]:
all_df_1.columns

Index(['stmt_mth', 'cycle_end_dt', 'vintage', 'actived_acct_cnt',
       'attrition_cnt', 'co_cnt', 'other_close_cnt', 'open_acct_cnt',
       'active_acct_cnt', 'active_rate', 'actived_credit_limit',
       'open_credit_limit', 'active_credit_limit', 'avg_active_credit_line',
       'transactor_acct_cnt', 'transactor_acct_pct', 'revolver_acct_cnt',
       'revolver_acct_pct', 'transactor_revolver_acct_cnt',
       'transactor_revolver_acct_pct', 'tot_end_os_bal_amt',
       'acct_end_os_bal_amt', 'tot_txn_amt', 'acct_txn_amt', 'txn_cl_util_pct',
       'tot_txn_cnt', 'acct_txn_cnt', 'txn_size_amt', 'tot_adb_amt',
       'acct_adb_amt', 'adb_cl_util_pct', 'tot_prncp_amt',
       'incremental_open_fee_acct_cnt', 'all_adb_amt', 'revolving_rate',
       'attrition_pct', 'co_pct', 'retention_pct', 'revolving_pct',
       'active_rate_2'],
      dtype='object')

In [7]:
actived_acct_cnt = pd.pivot_table(all_df_1,values='actived_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
actived_acct_cnt.to_csv("activated_acct_cnt_pivot.csv")

In [8]:
attrition_cnt = pd.pivot_table(all_df_1,values='attrition_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
attrition_cnt.to_csv("attrition_cnt.csv")

In [9]:
co_cnt = pd.pivot_table(all_df_1,values='co_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
co_cnt.to_csv("co_cnt.csv")

In [10]:
other_close_cnt = pd.pivot_table(all_df_1,values='other_close_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
other_close_cnt.to_csv("other_close_cnt.csv")

In [11]:
open_acct_cnt = pd.pivot_table(all_df_1,values='open_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
open_acct_cnt.to_csv("open_acct_cnt.csv")

In [12]:
all_df_1['retention_pct'] = all_df_1['open_acct_cnt']/ all_df_1['actived_acct_cnt']
retention_pct = pd.pivot_table(all_df_1,values='retention_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
retention_pct.iloc[-1] = open_acct_cnt.iloc[-1]/actived_acct_cnt.iloc[-1]
retention_pct.to_csv("retention_pct.csv")

In [13]:
actived_credit_limit = pd.pivot_table(all_df_1,values='actived_credit_limit',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
actived_credit_limit.to_csv("actived_credit_limit.csv")

In [15]:
open_credit_limit = pd.pivot_table(all_df_1,values='open_credit_limit',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
open_credit_limit.to_csv("open_credit_limit.csv")

In [14]:
active_credit_limit = pd.pivot_table(all_df_1,values='active_credit_limit',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
active_credit_limit.to_csv("active_credit_limit.csv")

In [17]:
active_acct_cnt = pd.pivot_table(all_df_1,values='active_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
active_acct_cnt.to_csv("active_acct_cnt.csv")

In [19]:
avg_active_credit_line = pd.pivot_table(all_df_1,values='avg_active_credit_line',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
avg_active_credit_line.iloc[-1] = active_credit_limit.iloc[-1]/active_acct_cnt.iloc[-1]
avg_active_credit_line.to_csv("avg_active_credit_line.csv")

In [20]:
active_rate = pd.pivot_table(all_df_1,values='active_rate',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
active_rate.iloc[-1] = active_acct_cnt.iloc[-1]/open_acct_cnt.iloc[-1]
active_rate.to_csv("active_rate.csv")

In [21]:
transactor_acct_cnt = pd.pivot_table(all_df_1,values='transactor_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
transactor_acct_cnt.to_csv("transactor_acct_cnt.csv")

In [22]:
transactor_acct_pct = pd.pivot_table(all_df_1,values='transactor_acct_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
transactor_acct_pct.iloc[-1] =  transactor_acct_cnt.iloc[-1]/open_acct_cnt.iloc[-1]
transactor_acct_pct.to_csv("transactor_acct_pct.csv")

In [23]:
revolver_acct_cnt = pd.pivot_table(all_df_1,values='revolver_acct_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
revolver_acct_cnt.to_csv("revolver_acct_cnt.csv")

In [24]:
revolver_acct_pct = pd.pivot_table(all_df_1,values='revolver_acct_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
revolver_acct_pct.iloc[-1] =  revolver_acct_cnt.iloc[-1]/open_acct_cnt.iloc[-1]
revolver_acct_pct.to_csv("revolver_acct_pct.csv")

In [25]:
tot_end_os_bal_amt = pd.pivot_table(all_df_1,values='tot_end_os_bal_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_end_os_bal_amt.to_csv("tot_end_os_bal_amt.csv")

In [26]:
acct_end_os_bal_amt = pd.pivot_table(all_df_1,values='acct_end_os_bal_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
acct_end_os_bal_amt.iloc[-1] = tot_end_os_bal_amt.iloc[-1]/active_acct_cnt.iloc[-1]
acct_end_os_bal_amt.to_csv("acct_end_os_bal_amt.csv")

In [57]:
tot_avg_os_bal_amt = pd.pivot_table(all_df_1,values='tot_avg_os_bal_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_avg_os_bal_amt.to_csv("tot_avg_os_bal_amt.csv")

KeyError: 'tot_avg_os_bal_amt'

In [92]:
acct_avg_os_bal_amt = pd.pivot_table(all_df_1,values='acct_avg_os_bal_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
acct_avg_os_bal_amt.iloc[-1] = tot_avg_os_bal_amt.iloc[-1]/active_acct_cnt.iloc[-1]  

KeyError: 'acct_avg_os_bal_amt'

In [93]:
cl_util_rate = pd.pivot_table(all_df_1,values='cl_util_rate',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
cl_util_rate.iloc[-1] = tot_avg_os_bal_amt.iloc[-1]/active_credit_limit.iloc[-1]

KeyError: 'cl_util_rate'

In [27]:
tot_txn_amt = pd.pivot_table(all_df_1,values='tot_txn_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_txn_amt.to_csv("tot_txn_amt.csv")

In [28]:
acct_txn_amt = pd.pivot_table(all_df_1,values='acct_txn_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
acct_txn_amt.iloc[-1] = tot_txn_amt.iloc[-1]/active_acct_cnt.iloc[-1]
acct_txn_amt.to_csv("acct_txn_amt.csv")

In [30]:
txn_cl_util_pct = pd.pivot_table(all_df_1,values='txn_cl_util_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
txn_cl_util_pct.iloc[-1] = tot_txn_amt.iloc[-1]/active_credit_limit.iloc[-1]
txn_cl_util_pct.to_csv("txn_cl_util_pct.csv")

In [31]:
tot_txn_cnt = pd.pivot_table(all_df_1,values='tot_txn_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_txn_cnt.to_csv("tot_txn_cnt.csv")

In [32]:
acct_txn_cnt = pd.pivot_table(all_df_1,values='acct_txn_cnt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
acct_txn_cnt.iloc[-1] = tot_txn_cnt.iloc[-1]/active_acct_cnt.iloc[-1]
acct_txn_cnt.to_csv("acct_txn_cnt.csv")

In [33]:
txn_size_amt = pd.pivot_table(all_df_1,values='txn_size_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
txn_size_amt.iloc[-1] = tot_txn_amt.iloc[-1]/tot_txn_cnt.iloc[-1]
txn_size_amt.to_csv("txn_size_amt.csv")

In [34]:
tot_adb_amt = pd.pivot_table(all_df_1,values='tot_adb_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_adb_amt.to_csv("tot_adb_amt.csv")

In [35]:
acct_adb_amt = pd.pivot_table(all_df_1,values='acct_adb_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
acct_adb_amt.iloc[-1] = tot_adb_amt.iloc[-1]/active_acct_cnt.iloc[-1]
acct_adb_amt.to_csv("acct_adb_amt.csv")

In [36]:
adb_cl_util_pct = pd.pivot_table(all_df_1,values='adb_cl_util_pct',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
adb_cl_util_pct.iloc[-1] = tot_adb_amt.iloc[-1]/active_credit_limit.iloc[-1]
adb_cl_util_pct.to_csv("adb_cl_util_pct.csv")

In [37]:
tot_prncp_amt = pd.pivot_table(all_df_1,values='tot_prncp_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
tot_prncp_amt.to_csv("tot_prncp_amt.csv")

In [38]:
all_adb = pd.pivot_table(all_df_1,values='all_adb_amt',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)

revolving_pct = pd.pivot_table(all_df_1,values='revolving_rate',index='vintage',columns='stmt_mth',aggfunc='sum',margins=True,margins_name = 'Total').drop('Total',axis=1)
revolving_pct.iloc[-1] = tot_adb_amt.iloc[-1]/all_adb.iloc[-1]
revolving_pct.to_csv("revolving_pct.csv")